In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

from openeew.data.aws import AwsDataClient
from openeew.data.df import get_df_from_records

# When importing pandas profiling matplotlib cannot show plots
# solved by importing matplotlib 
# after pandas_profiling and adding the magic command. 
# Magic command is required otherwise nothing is displayed:

import matplotlib.pyplot as plt
%matplotlib inline


In [6]:
data_client = AwsDataClient('cl')

In [7]:
devices = data_client.get_current_devices()

In [48]:
# Get earthquake location and datetime
# I chose one that occured in 2020

eq = {
    'latitude': -27.969,
    'longitude': -71.241,
    'date_utc': '2020-09-01 00:09:00'
    }

In [35]:
start_date_utc = '2020-08-31 23:50:00'
end_date_utc = '2020-09-01 02:00:00'

In [36]:
# Allow nested asyncio event loop
# See https://github.com/erdewit/nest_asyncio
import nest_asyncio
nest_asyncio.apply() 

In [37]:
records = data_client.get_filtered_records(start_date_utc, end_date_utc, device_ids=None)

In [38]:
#!pip install aioboto3==6.0.0

In [39]:
records_df = get_df_from_records(records)


In [40]:
records_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244256 entries, 0 to 31
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   country_code  244256 non-null  object 
 1   device_id     244256 non-null  object 
 2   x             244256 non-null  float64
 3   y             244256 non-null  float64
 4   z             244256 non-null  float64
 5   device_t      244256 non-null  float64
 6   cloud_t       244256 non-null  float64
 7   sr            244256 non-null  float64
 8   sample_t      244256 non-null  float64
dtypes: float64(7), object(2)
memory usage: 18.6+ MB


In [41]:
records_df.describe()

,x,y,z,device_t,cloud_t,sr,sample_t
count,244256.000000,244256.000000,244256.000000,2.442560e+05,2.442560e+05,244256.00,2.442560e+05
mean,-0.000136,-0.000026,0.000050,1.598922e+09,1.598922e+09,31.25,1.598922e+09
std,0.045775,0.043230,0.060903,2.251532e+03,2.251524e+03,0.00,2.251524e+03
min,-0.700000,-0.200000,-0.320000,1.598918e+09,1.598918e+09,31.25,1.598918e+09
25%,-0.030000,-0.030000,-0.040000,1.598920e+09,1.598920e+09,31.25,1.598920e+09
50%,0.000000,0.000000,0.000000,1.598922e+09,1.598922e+09,31.25,1.598922e+09
75%,0.030000,0.030000,0.040000,1.598924e+09,1.598924e+09,31.25,1.598924e+09
max,0.820000,0.250000,0.390000,1.598926e+09,1.598926e+09,31.25,1.598926e+09


In [42]:
records_df.head()

,country_code,device_id,x,y,z,device_t,cloud_t,sr,sample_t
0,cl,1003,0.00,0.03,-0.04,1.598918e+09,1.598918e+09,31.25,1.598918e+09
1,cl,1003,0.00,0.00,0.03,1.598918e+09,1.598918e+09,31.25,1.598918e+09
2,cl,1003,0.04,0.04,0.05,1.598918e+09,1.598918e+09,31.25,1.598918e+09
3,cl,1003,0.00,0.04,0.09,1.598918e+09,1.598918e+09,31.25,1.598918e+09
4,cl,1003,-0.06,-0.03,0.09,1.598918e+09,1.598918e+09,31.25,1.598918e+09


In [43]:
records_df.groupby("device_id")["x"].count()

device_id
1003    244256
Name: x, dtype: int64

In [49]:
import folium
# Create map centered at earthquake epicenter
m = folium.Map(
        location=[eq['latitude'], eq['longitude']],
        zoom_start=7
        )
# Add circle to indicate epicenter
folium.Circle(
    radius=10000,
    location=[eq['latitude'], eq['longitude']],
    color='crimson',
    fill='crimson',
    ).add_to(m)

In [25]:
import plotnine as pn
def plot_seismograms(device_id):
    
    # Get earthquake date as datetime.datetime object
    eq_dt = AwsDataClient._get_dt_from_str(eq['date_utc'])

    plots = []
    for axis in ['x', 'y', 'z']:
        plots.append(
            pn.ggplot(
                records_df[records_df['device_id'] == device_id],
                pn.aes('sample_dt', axis)
                ) + \
            pn.geom_line(color='blue') + \
            pn.scales.scale_x_datetime(
                date_breaks='1 minute',
                date_labels='%H:%M:%S'
                ) + \
            pn.geoms.geom_vline(
                xintercept=eq_dt,
                color='crimson'
                ) + \
            pn.labels.ggtitle(
                'device {}, axis {}'.format(
                      device_id, axis)
                      )
                )
    
    # Now output the plots    
    for p in plots:
        print(p)

In [26]:
plot_seismograms('1000')

NameError: name 'eq' is not defined